In [12]:
%load_ext autoreload
%autoreload 2

from time import sleep
import pandas as pd
from tqdm import tqdm
from scraper import get_program_links
from scraper import extract_program_details
import csv


# Base URL to handle relative links
BASE_URL = "https://www.foerderdatenbank.de/"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
start_url = "https://www.foerderdatenbank.de/SiteGlobals/FDB/Forms/Suche/Foederprogrammsuche_Formular.html?input_=23adddb0-dcf7-4e32-96f5-93aec5db2716&submit=Suchen&resourceId=0065e6ec-5c0a-4678-b503-b7e7ec435dfd&filterCategories=FundingProgram&pageLocale=de"

# check if there is a cache of the collected links (data/program_links.txt) and load from there.
try:
    with open("data/program_links.txt", "r") as f:
        program_links = f.read().splitlines()
except FileNotFoundError:
    program_links = get_program_links(url=start_url, BASE_URL=BASE_URL)

# Print the collected links
for link in program_links:
    print(link)

https://www.foerderdatenbank.de/FDB/Content/DE/Foerderprogramm/Land/Sachsen/buergschaft-sachsen-beteiligung.html
https://www.foerderdatenbank.de/FDB/Content/DE/Foerderprogramm/Land/NRW/garantien-von-beteiligungen-an-kmu-der-gewerbliche.html
https://www.foerderdatenbank.de/FDB/Content/DE/Foerderprogramm/Land/Sachsen/foerderrichtlinie-ausgleichszulage-sachsen.html
https://www.foerderdatenbank.de/FDB/Content/DE/Foerderprogramm/Land/Sachsen/rettung-umstrukturier-kleine-mittlere-unternehmen.html
https://www.foerderdatenbank.de/FDB/Content/DE/Foerderprogramm/Land/Sachsen/investitionen-teilhabe.html
https://www.foerderdatenbank.de/FDB/Content/DE/Foerderprogramm/Land/Sachsen/foerderung-der-kommunale-praevention-rl-komm-praev.html
https://www.foerderdatenbank.de/FDB/Content/DE/Foerderprogramm/Land/Sachsen/foerderung-sicherung-weiterentwickl-qualitaet-kita.html
https://www.foerderdatenbank.de/FDB/Content/DE/Foerderprogramm/Land/Thueringen/guw-thueringen.html
https://www.foerderdatenbank.de/FDB/C

In [4]:
# write the collected links to a file, so we don't have to scrape them again.
with open("data/program_links.txt", "w") as f:
    f.write("\n".join(program_links))

In [25]:
try:
    with open("data/funding_programs.csv", "r") as f:
        reader = csv.DictReader(f)
        program_details = list(reader)
        scraped_links = {row["url"] for row in program_details}
except FileNotFoundError:
    program_details = {}

for link in tqdm(program_links[:30]):
    if link in scraped_links:
        continue
    program_details[link] = extract_program_details(url=link)
    sleep(1)  # Be nice to the server

[{'url': 'https://www.foerderdatenbank.de/FDB/Content/DE/Foerderprogramm/Land/Sachsen/buergschaft-sachsen-beteiligung.html', 'name': 'Beteiligungsgarantien der Bürgschaftsbank Sachsen', 'kurzzusammenfassung': 'Kurztext\nWenn Sie als kleines oder mittleres Unternehmen für Beteiligungen von privaten Kapitalbeteiligungsgesellschaften eine Ausfallgarantie benötigen, kann diese unter bestimmten Voraussetzungen von der Bürgschaftsbank Sachsen übernommen werden.\nVolltext\nDie Bürgschaftsbank Sachsen übernimmt Ausfallgarantien für Beteiligungen von privaten Kapitalbeteiligungsgesellschaften an kleinen und mittleren Unternehmen in Sachsen, wenn die Beteiligungen ohne Garantien nicht oder nicht zu angemessenen Bedingungen zustande kommen würden.\nEs werden vor allem Garantien für\nübernommen.\nSie erhalten die Förderung als Ausfallgarantie. Die Beteiligung soll maximal EUR 1,25 Millionen (in begründeten Ausnahmefällen EUR 2,5 Millionen) betragen, wobei die Garantie 75 Prozent der Beteiligungssu

In [21]:
funding_programs = pd.DataFrame.from_dict(program_details, orient='index')

In [23]:
funding_programs.to_csv("data/funding_programs.csv", index_label="url")